# Advent of Code 2017, Dyalog APL edition

To see a correct render of this notebook, check it out on [nbviewer](https://nbviewer.jupyter.org/github/xpqz/AoCDyalog/blob/master/Advent%20of%20Code%202017%20Dyalog%20APL.ipynb).

Annotated solutions in Dyalog APL.

Note that part of the charm of AoC is that every user (or at least groups of users) gets their own unique data set. Some of the solutions below exploit quirks in my particular data set, and so may conceivably not work for the general case.

In [38]:
⎕IO←1
]box on -style=max -trains=tree -fns=on
]rows on

┌→─────────────────────────────────────┐
│Was ON -style=max -trains=tree -fns=on│
└──────────────────────────────────────┘

┌→──────┐
│Was OFF│
└───────┘

### Day 1: Inverse Captcha
https://adventofcode.com/2017/day/1

In [5]:
DAY1←⊃⊃⎕NGET'data/2017/01.txt'1

Tack on the first item to the end, as per the problem statement.

In [55]:
DATA←DAY1,1⌷DAY1

We use a length-2 windowed reduction using equality to find all location where item n is equal to item n+1 as a binary map. We convert this to position (`⍸`) and pick the corresponding items. As we're still dealing with characters, we need to convert to integers (`⍎¨`) and then just sum them up.

In [56]:
+/⍎¨DATA[⍸2=/DATA] ⍝ Part 1: 1341

1341

For part 2, we create a matrix where column 1 is the input and column 2 is the data rotated by half its length. Then compare the elements of each row to produce the binary map, and the rest is as per part 1.

In [17]:
+/⍎¨DAY1[⍸=/⍉↑(DAY1)(DAY1⊖⍨2÷⍨≢DAY1)] ⍝ Part 2: 1348

1348

### Day 2: Corruption Checksum
https://adventofcode.com/2017/day/2

In [52]:
DAY2←16 16⍴⍎¨'\d+'⎕S'&'⊢⊃⎕NGET'data/2017/02.txt'1

Part 1 -- sum max-min for every row

In [53]:
+/(⌈/-⌊/)DAY2 ⍝ Part 1: 39126

39126

In part 2 we're looking for the single pair of items per row that divides cleanly. For each row we generate all combinations, and try both ways of modular division, looking for a zero (not two) using 

    {≠/0=⍺(|,|⍨)⍵}
    
Finally, return the larger÷smaller and sum up everything.

In [79]:
Day2←{target←1↑⍸{≠/0=⍺(|,|⍨)⍵}/¨cmb←,⍵∘.,⍵⋄÷/pair[⍒pair←target⊃cmb]}

In [80]:
+/Day2¨↓DAY2 ⍝ Part 2: 258

258

### Day 3: Spiral Memory
https://adventofcode.com/2017/day/3

This [spiral](http://www.mathrecreation.com/2011/06/sequences-on-spiral.html) has a number of interesting properties. See http://oeis.org/A080335

We can exploit that we have perfect squares on the SE diagonal, which allows us to identify the size of the square which has the target number along one of its edges. We can then identify which edge, and count backwards from the next corner of the square (anti-clockwise).

In [83]:
⎕IO←0
DAY3←325489

In [82]:
Square←{r←⍵*0.5⋄⍵=2*⍨⌊r:r⋄0≠2|⌈r:⌈r⋄1+⌈r} ⍝ Smallest odd number < the square root, unless perfect square

In [110]:
]dinput
Day3p1←{
    val←⍵
    width←Square ⍵                    ⍝ The width of the square in which the value is found
    edge←⌊((width*2)-val)÷(width-1)   ⍝ The side of the square: S E N W ← 0 1 2 3
    pos←edge⊃,size,-size∘.,-size,size←⌊width÷2  ⍝ Grid coords of corner
    _←(edge⊃,¯1 0∘.,0 ¯1)∘{pos+←⍺⋄⍵-1}⍣{⍺=val} (width*2)-edge×width-1  ⍝ Sequence at the nearest corner, acw
    +/|¨pos                           ⍝ Manhattan distance to centre
}

In [111]:
Day3p1 DAY3 ⍝ 552

552

For part 2, we work our way outwards following the same spiral pattern, but this time the value of each point is the sum of the values of its 8-connected neighbours _at the time of visit_. The APL solution isn't pretty.

In [119]:
]dinput
Coords←{
    ⍝ Generate all coordinate pairs for the circumference of a square ⍵
    pos←(⌊⍵÷2),(-⌊⍵÷2)+1
    
    E←(⊂pos)+0,¨⍳⍵-1
    N←(¯1↑E)+(-⍳⍵),¨0
    W←(¯1↑N)+0,¨-⍳⍵
    S←(¯1↑W)+(⍳⍵),¨0
    ∪E, N, W, S
}

In [116]:
Neighbours←{(⊂⍵)+(0 1)(1 1)(1 0)(1 ¯1)(0 ¯1)(¯1 ¯1)(¯1 0)(¯1 1)} ⍝ 8-neighbours

In [117]:
]dinput
Day3p2←{
    target←⍵
    seen←,⊂0 0 ⋄ vals←,1
    { ⍝ Move outwards one square at a time (i.e. odd numbers)
        found←{ ⍝ Walk circumference, adding up values from visible neighbours
            0=≢⍵:0
            pos←⊃1↑⍵
            old←seen⍳Neighbours pos
            newVal←+/vals[((≢seen)>old)/old]
            newVal>target:newVal
            vals,←newVal
            seen,←⊂pos
            ∇1↓⍵
        } Coords ⍵
        found>0:found
        ∇⍵+2
    } 3
}

In [120]:
Day3p2 325489 ⍝ 330785

330785

### Day 4: High-Entropy Passphrases
https://adventofcode.com/2017/day/4

In [130]:
⎕IO←1
DAY4←' '(≠⊆⊢)¨⊃⎕NGET'data/2017/04.txt'1

Find phrases which don't match themselves with dupes removed.

In [142]:
+/(∪≡⊢)¨DAY4 ⍝ Part 1: 325

325

Part 2: apply the same process, but sort all words first.

In [151]:
+/(∪≡⊢)¨{⍵[⍋⍵]}¨¨DAY4 ⍝ Part 2: 119

119

### Day 5: A Maze of Twisty Trampolines, All Alike
https://adventofcode.com/2017/day/5

In [155]:
⎕IO←1
DAY5←⍎¨⊃⎕NGET'data/2017/05.txt'1

In [169]:
]dinput
Day5←{
    instr←⍵
    modifier←⍺⍺
    1 {
        ip←1↑⍵
        jmp←ip⌷instr
        (ip+jmp)>≢instr:⍺
        instr[ip]+←modifier jmp
        (⍺+1)∇ip+jmp
    } 1
}

In [167]:
{1} Day5 DAY5 ⍝ Part 1: 372671

372671

In [168]:
{⍵≥3:¯1⋄1} Day5 DAY5 ⍝ Part 2: 25608480 (takes a minute or so)

25608480

### Day 6: Memory Reallocation
https://adventofcode.com/2017/day/6

Part 1 and 2 differs only in the end condition and the start value. 

In [219]:
⎕IO←0
DAY6←2 8 8 5 4 2 3 1 5 5 1 2 15 13 5 14

In [229]:
]dinput
Redist←{
    (0@m⊢⍵) {                        ⍝ Zero out the current bin
        0=⊃⍵:⍺                       ⍝ Return end state
        (1∘+@n⊢⍺)∇(⊃⍵-1),n←16|1+⊢/⍵  ⍝ Move one index on, mod size and add 1. Decrease value to distribute.
    } ⍵[m],m←(⊢⍳⌈/)⍵                 ⍝ Value and index of the largest element
}

In [230]:
Day6p1←{seen←⊂⍵⋄e←Redist⍣{seen∊⍨⊂⍺:1⋄0⊣seen,←⊂⍺} ⍵⋄(⊂e),≢seen} ⍝ Stop if we've seen a state before.

In [231]:
(elem seen)←Day6p1 DAY6
seen ⍝ 3156

3156

In [232]:
Day6p2←{count←0⋄target←⍵⋄_←Redist⍣{count+←1⋄⍺≡target} ⍵⋄count} ⍝ Stop when we revisit end state of part 1.

In [233]:
Day6p2 elem ⍝ Part 2: 1610

1610